# Objetivo:
 - Plotar ocorrências de focos de calor do sensor MODIS num mapa interativo usando o [Leafmap](https://leafmap.org/)
   
# Acesso aos dados: 
- FIRMS: https://firms.modaps.eosdis.nasa.gov/active_fire/
- Shapefiles no formato Json: https://github.com/tbrugz/geodata-br

# Carrega os dados

In [69]:
# importa bibliotecas
import pandas as pd
import leafmap
import geopandas as gpd

# carrega shapefile dos munícipios do Brasil no formato Json
shapefile_municipios_brasil = 'https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-100-mun.json'

# carrega o dado do FIRMS de 24H
df_24h = pd.read_csv('https://firms.modaps.eosdis.nasa.gov/data/active_fire/modis-c6.1/csv/MODIS_C6_1_South_America_24h.csv')

# carrega o dado do FIRMS de 48H
df_48h = pd.read_csv('https://firms.modaps.eosdis.nasa.gov/data/active_fire/modis-c6.1/csv/MODIS_C6_1_South_America_48h.csv')

# carrega o dado do FIRMS de 7dias
df_7d = pd.read_csv('https://firms.modaps.eosdis.nasa.gov/data/active_fire/modis-c6.1/csv/MODIS_C6_1_South_America_7d.csv')

# transforma a data em índice
df_48h['acq_date'] = pd.to_datetime(df_48h['acq_date']) # transformar a coluna 'acq_date' em formato datetime
df_48h.set_index('acq_date', inplace=True) # definir a coluna 'acq_date' como índice do DataFrame

In [70]:
df_24h

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,confidence,version,bright_t31,frp,daynight
0,-8.09306,-43.15047,306.06,3.64,1.79,2024-07-04,7,T,67,6.1NRT,290.70,45.55,N
1,-8.00569,-44.36577,317.35,4.54,1.96,2024-07-04,7,T,95,6.1NRT,289.96,142.41,N
2,-7.99896,-44.33345,302.13,4.50,1.95,2024-07-04,7,T,47,6.1NRT,288.88,46.46,N
3,-8.00379,-44.37297,313.67,4.54,1.96,2024-07-04,7,T,88,6.1NRT,289.87,115.17,N
4,-7.28733,-43.78180,303.73,3.97,1.85,2024-07-04,7,T,57,6.1NRT,291.04,40.28,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,11.06312,-68.62839,323.23,1.65,1.26,2024-07-05,1444,T,77,6.1NRT,296.19,26.40,D
2009,10.05429,-64.86214,316.14,3.39,1.74,2024-07-05,1444,T,36,6.1NRT,295.32,43.31,D
2010,10.46557,-69.42281,316.99,1.47,1.20,2024-07-05,1444,T,74,6.1NRT,288.56,17.42,D
2011,10.46088,-69.42945,311.00,1.47,1.20,2024-07-05,1444,T,49,6.1NRT,287.31,8.85,D


In [71]:
df_48h

,latitude,longitude,brightness,scan,track,acq_time,satellite,confidence,version,bright_t31,frp,daynight
acq_date,,,,,,,,,,,,
2024-07-03,-25.20566,-42.87568,306.25,1.09,1.04,100,T,43,6.1NRT,293.60,6.36,N
2024-07-03,-24.65354,-42.51581,316.42,1.14,1.06,100,T,91,6.1NRT,294.41,15.85,N
2024-07-03,-22.91011,-43.73167,317.70,1.07,1.03,102,T,96,6.1NRT,292.09,17.72,N
2024-07-03,-22.91163,-43.74187,305.64,1.07,1.03,102,T,65,6.1NRT,292.09,7.66,N
2024-07-03,-22.59694,-48.04370,331.46,1.16,1.07,102,T,100,6.1NRT,288.45,40.89,N
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-05,11.06312,-68.62839,323.23,1.65,1.26,1444,T,77,6.1NRT,296.19,26.40,D
2024-07-05,10.05429,-64.86214,316.14,3.39,1.74,1444,T,36,6.1NRT,295.32,43.31,D
2024-07-05,10.46557,-69.42281,316.99,1.47,1.20,1444,T,74,6.1NRT,288.56,17.42,D


In [72]:
df_7d

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,confidence,version,bright_t31,frp,daynight
0,-23.51468,-50.10640,305.89,1.98,1.37,2024-06-28,56,T,66,6.1NRT,285.95,24.24,N
1,-23.51660,-50.12528,302.71,1.98,1.37,2024-06-28,56,T,51,6.1NRT,286.15,19.27,N
2,-22.05200,-47.11203,313.34,1.21,1.09,2024-06-28,56,T,87,6.1NRT,287.25,17.53,N
3,-22.07286,-49.28851,314.36,1.65,1.26,2024-06-28,56,T,89,6.1NRT,288.10,30.68,N
4,-21.11101,-43.25639,308.93,1.04,1.02,2024-06-28,56,T,76,6.1NRT,286.79,11.16,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9704,11.06312,-68.62839,323.23,1.65,1.26,2024-07-05,1444,T,77,6.1NRT,296.19,26.40,D
9705,10.05429,-64.86214,316.14,3.39,1.74,2024-07-05,1444,T,36,6.1NRT,295.32,43.31,D
9706,10.46557,-69.42281,316.99,1.47,1.20,2024-07-05,1444,T,74,6.1NRT,288.56,17.42,D
9707,10.46088,-69.42945,311.00,1.47,1.20,2024-07-05,1444,T,49,6.1NRT,287.31,8.85,D


# Plota mapa para `América do Sul`

In [73]:
Map = leafmap.Map(center=(0, -60), zoom=4)

Map.add_basemap('HYBRID')

data_atual = f'{df_48h.index[-1].year}-{str(df_48h.index[-1].month).zfill(2)}-{str(df_48h.index[-1].day).zfill(2)}'

Map.add_xy_data(df_48h.loc[data_atual], x="longitude", y="latitude")

Map.add_geojson(shapefile_municipios_brasil)

Map

Map(center=[0, -60], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

# Plota mapa para `Brasil`

## Selciona os focos apenas do Brasil

In [74]:
# leitura de arquivo
shapefile_municipios_brasil_gpd = gpd.read_file(shapefile_municipios_brasil)

# novo geodataframe
queimadas_48h = gpd.GeoDataFrame(df_48h.reset_index(), geometry=gpd.points_from_xy(df_48h.longitude, df_48h.latitude))

# atribuição do sistema de coordenadas de referência (CRS)
queimadas_48h.crs = shapefile_municipios_brasil_gpd.crs

# aplicando a mascara
queimadas_brasil_48h = gpd.overlay(queimadas_48h, shapefile_municipios_brasil_gpd, how='intersection')

# transformando a coluna data para índice do dataframe
queimadas_brasil_48h['acq_date'] = pd.to_datetime(queimadas_brasil_48h['acq_date'])
queimadas_brasil_48h.set_index('acq_date', inplace=True)

# mostra o dado
queimadas_brasil_48h.tail()

,latitude,longitude,brightness,scan,track,acq_time,satellite,confidence,version,bright_t31,frp,daynight,id,name,description,geometry
acq_date,,,,,,,,,,,,,,,,
2024-07-05,-23.16136,-54.18064,313.93,1.09,1.04,1316,T,51,6.1NRT,295.93,7.06,D,5005707,Naviraí,Naviraí,POINT (-54.18064 -23.16136)
2024-07-05,-23.26119,-53.97595,322.33,1.11,1.05,1316,T,68,6.1NRT,299.07,15.23,D,5004601,Itaquiraí,Itaquiraí,POINT (-53.97595 -23.26119)
2024-07-05,-23.26259,-53.96531,316.60,1.11,1.05,1316,T,42,6.1NRT,295.58,8.73,D,5004601,Itaquiraí,Itaquiraí,POINT (-53.96531 -23.26259)
2024-07-05,-25.23832,-50.58409,301.18,1.87,1.34,1316,T,45,6.1NRT,290.67,9.70,D,4110102,Imbituva,Imbituva,POINT (-50.58409 -25.23832)
2024-07-05,-25.52849,-51.98838,315.30,1.51,1.21,1316,T,60,6.1NRT,288.34,19.10,D,4104428,Candói,Candói,POINT (-51.98838 -25.52849)


## Plota mapa

In [75]:
Map = leafmap.Map(center=(0, -60), zoom=4)

Map.add_basemap('HYBRID')

data_atual = f'{queimadas_brasil_48h.index[-1].year}-{str(queimadas_brasil_48h.index[-1].month).zfill(2)}-{str(queimadas_brasil_48h.index[-1].day).zfill(2)}'

Map.add_xy_data(queimadas_brasil_48h[data_atual:data_atual], x="longitude", y="latitude")

#Map.add_geojson(brasil)

Map

Map(center=[0, -60], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…